In [16]:
import os
import numpy as np
import mne
import sklearn
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [17]:
def load_data_from_folder(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.edf'):
            filepath = os.path.join(folder_path, filename)
            eeg_data = mne.io.read_raw_edf(filepath, preload=True)
            data.append(eeg_data)
    return data
    
seizure_files = load_data_from_folder('Seizure/')
without_seizure_files = load_data_from_folder('No Seizure/')

Extracting EDF parameters from /Users/jes/Downloads/Justin/Seizure/chb01_15.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/hs/gpg0mmfd6mx223gdbgx0qkj40000gn/T/ipykernel_3170/596422966.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  eeg_data = mne.io.read_raw_edf(filepath, preload=True)


Extracting EDF parameters from /Users/jes/Downloads/Justin/Seizure/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/hs/gpg0mmfd6mx223gdbgx0qkj40000gn/T/ipykernel_3170/596422966.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  eeg_data = mne.io.read_raw_edf(filepath, preload=True)


Extracting EDF parameters from /Users/jes/Downloads/Justin/Seizure/chb01_16.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


...

/var/folders/hs/gpg0mmfd6mx223gdbgx0qkj40000gn/T/ipykernel_3170/596422966.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  eeg_data = mne.io.read_raw_edf(filepath, preload=True)


Extracting EDF parameters from /Users/jes/Downloads/Justin/No Seizure/chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/hs/gpg0mmfd6mx223gdbgx0qkj40000gn/T/ipykernel_3170/596422966.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  eeg_data = mne.io.read_raw_edf(filepath, preload=True)


Extracting EDF parameters from /Users/jes/Downloads/Justin/No Seizure/chb01_13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/var/folders/hs/gpg0mmfd6mx223gdbgx0qkj40000gn/T/ipykernel_3170/596422966.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  eeg_data = mne.io.read_raw_edf(filepath, preload=True)


In [18]:
import pandas as pd
def raw_to_dataframe(raw):
    eeg_data, times = raw[:, :]
    channels = raw.info['ch_names']
    df = pd.DataFrame(eeg_data.T, columns=channels)
    return df
seizure_dfs = [raw_to_dataframe(raw) for raw in seizure_files]
for df in seizure_dfs:
    df['Seizure'] = 1
without_seizure_dfs = [raw_to_dataframe(raw) for raw in without_seizure_files]
for df in without_seizure_dfs:
    df['Seizure'] = 0

In [19]:
all_dfs = seizure_dfs + without_seizure_dfs
final_df = pd.concat(all_dfs, ignore_index=True)
final_df

,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,F4-C4,...,T8-P8-0,P8-O2,FZ-CZ,CZ-PZ,P7-T7,T7-FT9,FT9-FT10,FT10-T8,T8-P8-1,Seizure
0,3.457875e-05,-7.951160e-05,1.010012e-04,2.623687e-04,1.191697e-05,1.426129e-05,1.119414e-04,1.803175e-04,-5.665446e-06,6.349206e-05,...,7.365079e-05,7.163858e-04,7.443223e-05,1.932112e-04,-1.006105e-04,3.350427e-04,-2.170452e-04,3.321123e-06,7.365079e-05,1
1,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1
2,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,5.860806e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,-1.953602e-07,1.953602e-07,1
3,1.953602e-07,-1.953602e-07,1.953602e-07,-1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,-1.953602e-07,-1.953602e-07,1.953602e-07,...,1.953602e-07,6.056166e-06,1.953602e-07,5.860806e-07,1.953602e-07,1.367521e-06,1.953602e-07,-1.758242e-06,1.953602e-07,1
4,1.953602e-07,1.953602e-07,1.953602e-07,5.860806e-07,1.953602e-07,1.953602e-07,1.953602e-07,9.768010e-07,1.953602e-07,1.953602e-07,...,1.953602e-07,5.860806e-07,1.953602e-07,-1.953602e-07,1.953602e-07,1.953602e-07,1.953602e-07,1.367521e-06,1.953602e-07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7372795,-4.512821e-05,2.715507e-05,1.504274e-05,9.181929e-06,3.711844e-06,-1.621490e-05,1.738706e-05,2.539683e-06,-6.056166e-06,-2.559219e-05,...,-5.177045e-05,-3.067155e-05,-2.012210e-05,-1.953602e-07,-1.465201e-05,-1.191697e-05,1.385104e-04,-3.887668e-05,-5.177045e-05,0
7372796,-4.043956e-05,2.402930e-05,8.791209e-06,7.228327e-06,1.191697e-05,-1.934066e-05,3.321123e-06,5.274725e-06,-6.837607e-06,-2.481074e-05,...,-5.958486e-05,-2.520147e-05,-1.894994e-05,-2.148962e-06,-8.400488e-06,-7.228327e-06,1.303053e-04,-3.457875e-05,-5.958486e-05,0
7372797,-3.809524e-05,2.051282e-05,-1.953602e-07,8.400488e-06,1.543346e-05,-2.207570e-05,-2.148962e-06,1.953602e-07,-4.493284e-06,-2.598291e-05,...,-5.880342e-05,-1.894994e-05,-2.051282e-05,-2.148962e-06,5.860806e-07,1.953602e-07,1.178022e-04,-3.692308e-05,-5.880342e-05,0
7372798,-4.161172e-05,1.777778e-05,-1.113553e-05,3.321123e-06,1.113553e-05,-2.481074e-05,-9.572650e-06,-7.619048e-06,-1.758242e-06,-2.715507e-05,...,-5.645910e-05,-4.786325e-05,-2.129426e-05,-6.446886e-06,1.152625e-05,-5.860806e-07,1.174115e-04,-4.278388e-05,-5.645910e-05,0


In [20]:
final_df.shape

(7372800, 24)

### CNN

In [21]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense,LSTM,Conv1D,MaxPooling1D
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
X = final_df.drop(columns=['Seizure'])  
y = final_df['Seizure']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) 

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_reshaped = X_train_scaled.reshape(-1, X_train.shape[1], 1)
X_val_reshaped = X_val_scaled.reshape(-1, X_val.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(-1, X_test.shape[1], 1)

In [23]:
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_reshaped, y_train, validation_data=(X_val_reshaped, y_val), epochs=5, batch_size=32)

loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/5
138240/138240 [==============================] - 413s 3ms/step - loss: 0.5084 - accuracy: 0.7541 - val_loss: 0.4946 - val_accuracy: 0.7629
Epoch 2/5
138240/138240 [==============================] - 370s 3ms/step - loss: 0.4923 - accuracy: 0.7640 - val_loss: 0.4916 - val_accuracy: 0.7639
Epoch 3/5
138240/138240 [==============================] - 377s 3ms/step - loss: 0.4879 - accuracy: 0.7663 - val_loss: 0.4882 - val_accuracy: 0.7656
Epoch 4/5
138240/138240 [==============================] - 412s 3ms/step - loss: 0.4851 - accuracy: 0.7677 - val_loss: 0.4872 - val_accuracy: 0.7664
Epoch 5/5
46080/46080 [==============================] - 72s 2ms/step - loss: 0.4905 - accuracy: 0.7635
Test Accuracy: 0.7635030150413513


In [24]:
y_pred = model.predict(X_test_reshaped)
y_pred_binary = (y_pred > 0.6).astype(int)
print(classification_report(y_test, y_pred_binary))

46080/46080 [==============================] - 65s 1ms/step
              precision    recall  f1-score   support

           0       0.76      0.78      0.77    737505
           1       0.77      0.76      0.77    737055

    accuracy                           0.77   1474560
   macro avg       0.77      0.77      0.77   1474560
weighted avg       0.77      0.77      0.77   1474560



### LSTM

In [26]:
from tensorflow.keras.layers import LSTM, Dropout
model = Sequential([
    LSTM(units=32, input_shape=(X_train.shape[1], 1), return_sequences=True),
    Dropout(0.2),
    LSTM(units=32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_reshaped, y_train, validation_data=(X_val_reshaped, y_val), epochs=5, batch_size=128)

loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", accuracy)

y_pred = model.predict(X_test_reshaped)
y_pred_binary = (y_pred > 0.6).astype(int)
print(classification_report(y_test, y_pred_binary))

Epoch 1/5
34560/34560 [==============================] - 507s 15ms/step - loss: 0.5598 - accuracy: 0.7150 - val_loss: 0.5313 - val_accuracy: 0.7375
Epoch 2/5
34560/34560 [==============================] - 511s 15ms/step - loss: 0.5304 - accuracy: 0.7396 - val_loss: 0.5199 - val_accuracy: 0.7454
Epoch 3/5
34560/34560 [==============================] - 507s 15ms/step - loss: 0.5196 - accuracy: 0.7473 - val_loss: 0.5077 - val_accuracy: 0.7542
Epoch 4/5
34560/34560 [==============================] - 521s 15ms/step - loss: 0.5108 - accuracy: 0.7533 - val_loss: 0.5010 - val_accuracy: 0.7588
Epoch 5/5
46080/46080 [==============================] - 178s 4ms/step - loss: 0.4962 - accuracy: 0.7624
Test Accuracy: 0.7624281048774719
46080/46080 [==============================] - 174s 4ms/step
              precision    recall  f1-score   support

           0       0.74      0.79      0.77    737505
           1       0.78      0.73      0.75    737055

    accuracy                           0.76 